### Required Imports

In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

c:\Anaconda\envs\Lab1\lib\site-packages\tensorflow\python\framework\dtypes.py:585: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [28]:
def load_and_unpickle(class_type, directory, subdirectory):
    """
    Description:
    Loads and Unpickles a file from the supplied parameters
    
    Parameter:
    class_type (str): class type to be loaded and unpickled\n
    directory (str): directory within the CWD\n
    subdirectory (str): subdirectory within the directory specified\n
    
    Returns:
    Specified dataframe if found in the directory\n
    None if not found\n
    """
    dir_path = f"{directory}/{subdirectory}"
    filename = f'{class_type}.pkl'
    filepath = os.path.join(dir_path, filename)

    if os.path.exists(filepath):
        df = pd.read_pickle(filepath)
        print(f"Loaded from {subdirectory} for class {class_type}")
        return df
    else:
        print(f"File not found: {filepath}")
        return None

In [29]:
def get_dataframes(clean=False):
    classes = ["Jump", "Run", "Walk", "Squat"]
    original_directory = "Original"
    clean_directory = "Clean"
    training_subdirectory = "Training"
    test_subdirectory = "Test"
    validation_subdirectory = "Validation"
    
    train_df_arr = []
    test_df_arr = []
    valid_df_arr = []
    
    complete_train_df = pd.DataFrame()
    complete_test_df = pd.DataFrame()
    complete_valid_df = pd.DataFrame()
    
    for class_type in classes:
        if(clean):
            train_df_arr.append(load_and_unpickle(class_type, clean_directory, training_subdirectory))
            test_df_arr.append(load_and_unpickle(class_type, clean_directory, test_subdirectory))
            valid_df_arr.append(load_and_unpickle(class_type, clean_directory, validation_subdirectory))
        else:
            train_df_arr.append(load_and_unpickle(class_type, original_directory, training_subdirectory))
            test_df_arr.append(load_and_unpickle(class_type, original_directory, test_subdirectory))
            valid_df_arr.append(load_and_unpickle(class_type, original_directory, validation_subdirectory))
    
    for train_df, test_df, valid_df in zip(train_df_arr, test_df_arr, valid_df_arr):
        complete_train_df = pd.concat([complete_train_df, train_df], ignore_index=True)
        complete_test_df = pd.concat([complete_test_df, test_df], ignore_index=True)
        complete_valid_df = pd.concat([complete_valid_df, valid_df], ignore_index=True)

    return complete_train_df, complete_test_df, complete_valid_df

In [70]:
original_train_df, original_test_df, original_valid_df = get_dataframes()
clean_train_df, clean_test_df, clean_valid_df = get_dataframes(True)

Loaded from Training for class Jump
Loaded from Test for class Jump
Loaded from Validation for class Jump
Loaded from Training for class Run
Loaded from Test for class Run
Loaded from Validation for class Run
Loaded from Training for class Walk
Loaded from Test for class Walk
Loaded from Validation for class Walk
Loaded from Training for class Squat
Loaded from Test for class Squat
Loaded from Validation for class Squat
Loaded from Training for class Jump
Loaded from Test for class Jump
Loaded from Validation for class Jump
Loaded from Training for class Run
Loaded from Test for class Run
Loaded from Validation for class Run
Loaded from Training for class Walk
Loaded from Test for class Walk
Loaded from Validation for class Walk
Loaded from Training for class Squat
Loaded from Test for class Squat
Loaded from Validation for class Squat
         ax        ay        az        gx        gy        gz Class
0  0.103585 -9.798624 -0.484243 -0.017339 -0.057202  0.023274  Jump
1  0.101190 -9.7

### Create Neural Network

In [32]:
def create_gru_neural_network(neurons, hidden_layers, timesteps):
    model = Sequential()
    # Input Layer with first hidden layer
    model.add(GRU(neurons, input_shape=(timesteps, 6), activation='tanh', return_sequences=True, name="Hidden_Layer_1_GRU"))
    # Add Hidden layers
    for i in range(0, hidden_layers - 1):
        model.add(GRU(neurons, activation='tanh', name=f"Hidden_layer_{i+2}_GRU", return_sequences=True))
   
    model.add(GRU(neurons, activation='tanh', name=f"Hidden_layer_{hidden_layers+1}_GRU"))
    # Output Layer
    model.add(Dense(4, activation='softmax', name="Output_Layer"))
    # Compile Model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [58]:
# def train_gru_model(train_df, valid_df, neurons, hidden_layers, timesteps, batch_size=32, epochs=50, normalize=False):
#         # Prepare features and labels
#     X_train = train_df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values  # Feature columns
#     Y_train = train_df['Class'].values  # Label column
#     X_val = valid_df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values # Feature columns
#     Y_val = valid_df['Class'].values  # Label column
    
#     # Normalize data if required
#     if normalize:
#         scaler = MinMaxScaler()
#         X_train = scaler.fit_transform(X_train)
#         X_val = scaler.transform(X_val)
#     # Reshape input data to be 3D [samples, timesteps, features]
#     # Here, we assume that you want to create a sequence of 'timesteps' for each sample
#     num_samples = X_train.shape[0] // timesteps  # Number of complete sequences
#     X_train = X_train[:num_samples * timesteps].reshape(num_samples, timesteps, 6)  # Reshape to 3D
#     Y_train = Y_train[: num_samples * timesteps]  # Corresponding labels

#     num_val_samples = X_val.shape[0] // timesteps  # Number of complete validation sequences
#     X_val = X_val[:num_val_samples * timesteps].reshape(num_val_samples, timesteps, 6)  # Reshape to 3D
#     Y_val = Y_val[: num_val_samples * timesteps]  # Corresponding validation labels
#     # Encode labels
#     label_encoder = LabelEncoder()
#     Y_train_encoded = label_encoder.fit_transform(Y_train)
#     Y_val_encoded = label_encoder.transform(Y_val)
    
#     # Create GRU model
#     model = create_gru_neural_network(neurons=neurons, hidden_layers=hidden_layers, timesteps=timesteps)
    
#     # Train the model
#     history = model.fit(
#         x=X_train,  # Reshape for GRU input
#         y=Y_train_encoded,
#         batch_size=batch_size,
#         epochs=epochs,
#         verbose=1,
#         validation_data=(X_val, Y_val_encoded)
#     )
    
#     return model, history, label_encoder

In [1]:
def train_gru_model(train_df, valid_df, neurons, hidden_layers, timesteps, batch_size=32, epochs=50, normalize=False):
    # Prepare features and labels
    X_train = train_df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values  # Feature columns
    Y_train = train_df['Class'].values  # Label column
    X_val = valid_df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values # Feature columns
    Y_val = valid_df['Class'].values  # Label column

    # Normalize data if required
    if normalize:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Reshape input data to be 3D [samples, timesteps, features]
    num_samples = X_train.shape[0] // timesteps  # Number of complete sequences
    X_train = X_train[:num_samples * timesteps].reshape(num_samples, timesteps, 6)  # Reshape to 3D
    Y_train = Y_train[:num_samples * timesteps]  # Corresponding labels
    
    # Make sure Y_train has the same number of entries as X_train's samples
    Y_train = Y_train[:num_samples]  # Adjust to only the number of sequences

    num_val_samples = X_val.shape[0] // timesteps  # Number of complete validation sequences
    X_val = X_val[:num_val_samples * timesteps].reshape(num_val_samples, timesteps, 6)  # Reshape to 3D
    Y_val = Y_val[:num_val_samples * timesteps]  # Corresponding validation labels

    # Make sure Y_val has the same number of entries as X_val's samples
    Y_val = Y_val[:num_val_samples]  # Adjust to only the number of sequences

    # Encode labels
    label_encoder = LabelEncoder()
    Y_train_encoded = label_encoder.fit_transform(Y_train)
    Y_val_encoded = label_encoder.transform(Y_val)

    # Create GRU model
    model = create_gru_neural_network(neurons=neurons, hidden_layers=hidden_layers, timesteps=timesteps)
    # # Train the model
    with tf.device('/GPU:0'):
        history = model.fit(
            x=X_train,  # Reshape for GRU input
            y=Y_train_encoded,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(X_val, Y_val_encoded)
        )
    
    return model, history, label_encoder


In [2]:
model, history, label_encoder = train_gru_model(
    train_df=original_train_df,
    valid_df=original_valid_df,
    neurons=10,
    hidden_layers=1,
    timesteps=300
    )

NameError: name 'original_train_df' is not defined